# Explore participant data

> This notebook explains how to explore phenotypic data table retrieve fields

- runtime: 10min 
- recommended instance: mem1_ssd1_v2_x8
- cost: <£0.10

This notebook depends on:
* **A Spark instance**

In this notebook, we will dive deeper into the phenotypic data stored in the Spark database.
We will retrieve the information about the fields, and learn how to get field id, title, and link to the UK Biobank Showcase, which provides more details and basic statistics about field.

## Import `dxdata` package and initialize Spark engine
### Docs at: https://github.com/dnanexus/OpenBio/blob/master/dxdata/getting_started_with_dxdata.ipynb

In [1]:
import dxdata
import os

# Initialize dxdata engine
engine = dxdata.connect(dialect="hive+pyspark")

## Connect to the dataset

Next, we can set a `DATASET_ID` variable, which takes a value: `[projectID]:[dataset ID]`
We use it to define the `dataset` with `dxdata.load_dataset` function.

**projectID** and **dataset ID** values are unique to your project.
Notebook example **101** explains how to get them.

In [ ]:
project = os.getenv('DX_PROJECT_CONTEXT_ID')
record = os.popen("dx find data --type Dataset --delimiter ',' | awk -F ',' '{print $5}'").read().rstrip()
DATASET_ID = project + ":" + record
dataset = dxdata.load_dataset(id=DATASET_ID)

pheno = dataset['participant']

## Extract field names from ukb_field_mapping.json

Now we'll load the field mapping JSON file and extract all field names in the format required for data extraction.

In [ ]:
import json

# Load the field mapping JSON file
# Update this path to match your actual file location
mapping_file = 'DNAnexus:ukb_field_mapping.json'

with open(mapping_file, 'r') as f:
    field_mapping = json.load(f)

# Extract all field IDs and convert to the format used in UK Biobank (prefixed with 'p')
field_names = []
for category, fields in field_mapping.items():
    for field_id, description in fields.items():
        field_name = f'p{field_id}'
        field_names.append(field_name)

# Remove duplicates and sort
field_names = sorted(list(set(field_names)))

print(f"Total unique fields to extract: {len(field_names)}")
print("\nField names:")
for field in field_names:
    print(field)


## Verify fields exist in the dataset and get their titles

Let's verify that these fields exist in the UK Biobank dataset and retrieve their full titles.


In [ ]:
# Verify fields and get their information (including ALL instances)
# Use fuzzy matching to find all instances of each field (e.g., p50, p50_i0, p50_i1, etc.)

verified_fields = []
missing_fields = []

for field_name in field_names:
    try:
        # Try exact match first (for fields without instances)
        try:
            field = pheno.find_field(name=field_name)
            verified_fields.append({
                'name': field.name,
                'title': field.title,
                'linkout': field.linkout
            })
            print(f"✓ [{field.name}]\t{field.title}")
        except:
            # If exact match fails, search for all instances using regex
            # Pattern matches: p50$ (exact) OR p50_i0, p50_i1, p50_i2, etc.
            pattern = f"^{field_name}$|^{field_name}_i\\d+$"
            instance_fields = list(pheno.find_fields(name_regex=pattern))
            
            if instance_fields:
                for field in instance_fields:
                    verified_fields.append({
                        'name': field.name,
                        'title': field.title,
                        'linkout': field.linkout
                    })
                    print(f"✓ [{field.name}]\t{field.title}")
                print(f"  → Found {len(instance_fields)} instances for {field_name}")
            else:
                raise Exception(f"No instances found")
                
    except Exception as e:
        missing_fields.append(field_name)
        print(f"✗ [{field_name}]\tNot found: {str(e)}")

print(f"\n\nSummary:")
print(f"Found: {len(verified_fields)} fields (including all instances)")
print(f"Missing: {len(missing_fields)} base fields")
if missing_fields:
    print(f"\nMissing fields: {missing_fields}")


## Save field names to JSON file

Save the verified field names to a JSON file in the same format as the original field names.json file.


In [ ]:
# Save field names to JSON file
output_file = '../biomni_logs_session_e1b5e60b-517b-48fb-a8df-486707b241bd_20251023_114557/extracted_field_names.json'

# Save as list (one field per line like the original)
with open(output_file, 'w') as f:
    json.dump([f['name'] for f in verified_fields], f, indent=0)

print(f"Field names saved to: {output_file}")
print(f"Total fields saved: {len(verified_fields)}")

# Also save detailed information
detail_output_file = '../biomni_logs_session_e1b5e60b-517b-48fb-a8df-486707b241bd_20251023_114557/field_details.json'
with open(detail_output_file, 'w') as f:
    json.dump(verified_fields, f, indent=2)

print(f"Field details saved to: {detail_output_file}")


## Extract data for respiratory disease cohort

This cell demonstrates how to extract the actual data for these fields. This may take some time depending on the dataset size.


In [ ]:
# Get field objects for extraction
field_objects = []
for field_info in verified_fields:
    try:
        field = pheno.find_field(name=field_info['name'])
        field_objects.append(field)
    except Exception as e:
        print(f"Error loading field {field_info['name']}: {e}")

# Add participant ID
field_objects.insert(0, pheno.find_field(name="eid"))

# Ensure we have ICD-10 diagnosis fields for filtering
# These fields contain the diagnosis codes
icd10_fields = ['p41202', 'p41204', 'p41270']
for icd_field in icd10_fields:
    try:
        # Check if already in field_objects
        if not any(f.name == icd_field for f in field_objects):
            field = pheno.find_field(name=icd_field)
            field_objects.append(field)
            print(f"Added ICD-10 diagnosis field: {icd_field}")
    except Exception as e:
        print(f"Warning: Could not add {icd_field}: {e}")

print(f"\nExtracting data for {len(field_objects)} fields...")

# Retrieve data (returns PySpark DataFrame)
spark_df = pheno.retrieve_fields(fields=field_objects, engine=engine)

# Get row count using PySpark method
row_count = spark_df.count()
col_count = len(spark_df.columns)
print(f"Data extracted: {row_count} participants, {col_count} fields")

# Convert to pandas DataFrame for easier manipulation
print("\nConverting to pandas DataFrame...")
df = spark_df.toPandas()

# Now you can use pandas methods
print(f"Full dataset shape: {df.shape}")
print("\nFirst few rows:")
print(df.head())

## Step 3: Convert to Natural Language Text

Now that we have extracted and filtered the respiratory cohort, we can convert the structured data to natural language text using the `process_respiratory_cohort.py` script.

This step will:
1. Read the CSV file with respiratory patients
2. Convert each patient's structured data to narrative clinical text
3. Save individual patient text files and a summary JSON

The natural text output is ready for use with large language models and multimodal foundation models.


In [ ]:
# Option 1: Import and use the converter directly in the notebook
import sys
sys.path.append('.')

from process_respiratory_cohort import RespiratoryPatientTextConverter

# Initialize converter
converter = RespiratoryPatientTextConverter(
    input_csv='ukb_respiratory_cohort.csv',
    output_dir='./processed_respiratory_patients'
)

# Process first 10 patients as a test
print("Processing respiratory cohort to natural language text...")
converter.process_cohort(max_patients=10)

print("\nProcessing complete! Check the output directory for results.")


### Alternative: Run as command-line script

You can also run the processing script from the command line:


In [ ]:
# Run for all patients
python process_respiratory_cohort.py \
    --input-csv ukb_respiratory_cohort.csv \
    --output-dir ./processed_respiratory_patients

# Or run for first 100 patients only
python process_respiratory_cohort.py \
    --input-csv ukb_respiratory_cohort.csv \
    --output-dir ./processed_respiratory_patients \
    --max-patients 100


### View Sample Natural Language Output

Let's look at an example of the converted natural language text:


In [ ]:
import os
import glob

# Find the first processed patient file
processed_dir = './processed_respiratory_patients'
if os.path.exists(processed_dir):
    patient_files = glob.glob(os.path.join(processed_dir, 'patient_*.txt'))
    
    if patient_files:
        # Read and display the first patient's natural text
        with open(patient_files[0], 'r') as f:
            sample_text = f.read()
        
        print("=" * 80)
        print("SAMPLE NATURAL LANGUAGE OUTPUT")
        print("=" * 80)
        print(sample_text)
        print("\n" + "=" * 80)
        print(f"\nTotal patients processed: {len(patient_files)}")
        print(f"Files saved to: {processed_dir}")
    else:
        print("No patient files found. Run the processing cell above first.")
else:
    print(f"Output directory not found: {processed_dir}")
    print("Run the processing cell above first.")


## Filter for Respiratory Disease Cohort

We will filter participants to include only those with respiratory disease diagnoses based on ICD-10 codes:
- **J09-J98**: Diseases of the respiratory system
- **I26-I27**: Pulmonary heart disease and diseases of pulmonary circulation

This filtering will be applied after data extraction using the diagnosis fields (p41202, p41204, p41270).


In [ ]:
import re

def has_respiratory_diagnosis(row):
    """
    Check if a participant has respiratory disease diagnosis based on ICD-10 codes:
    - J09-J98: Diseases of the respiratory system
    - I26-I27: Pulmonary heart disease and diseases of pulmonary circulation
    
    Note: ICD-10 codes may be stored without decimal points (e.g., J181 = J18.1)
    """
    # Collect all diagnosis codes from available ICD-10 fields
    all_diagnoses = []
    
    for field in ['p41202', 'p41204', 'p41270']:
        if field in row.index and row[field] is not None:
            # Handle both list and single string values
            if isinstance(row[field], list):
                all_diagnoses.extend(row[field])
            else:
                all_diagnoses.append(str(row[field]))
    
    # Check each diagnosis code
    for code in all_diagnoses:
        if code and isinstance(code, str):
            # Remove any whitespace and convert to uppercase
            code = code.strip().upper()
            
            # Check for J09-J98 (respiratory diseases)
            # ICD-10 format: Letter + 2-3 digits (category) + optional subcategory
            # Examples: J18.1 (stored as J181), J96.90 (stored as J9690)
            if code.startswith('J'):
                # Extract the category code (first 2 digits after 'J')
                # For J codes, category is always 2 digits: J00-J99
                match = re.match(r'J(\d{2})', code)
                if match:
                    category = int(match.group(1))
                    if 9 <= category <= 98:
                        return True
            
            # Check for I26-I27 (pulmonary heart disease)
            # For I codes, category is also 2 digits: I00-I99
            elif code.startswith('I'):
                match = re.match(r'I(\d{2})', code)
                if match:
                    category = int(match.group(1))
                    if 26 <= category <= 27:
                        return True
    
    return False

# Apply filter to get respiratory disease cohort
print("Filtering for respiratory disease patients...")
df['has_respiratory_disease'] = df.apply(has_respiratory_diagnosis, axis=1)

respiratory_df = df[df['has_respiratory_disease'] == True].copy()
respiratory_df = respiratory_df.drop(columns=['has_respiratory_disease'])

print(f"\nCohort filtering results:")
print(f"Original dataset: {len(df)} participants")
print(f"Respiratory disease cohort: {len(respiratory_df)} participants ({len(respiratory_df)/len(df)*100:.2f}%)")

# Show some diagnosis examples
print("\nExample diagnoses from filtered cohort (first 5 participants):")
for idx, row in respiratory_df.head().iterrows():
    diagnoses = []
    for field in ['p41202', 'p41204', 'p41270']:
        if field in row.index and row[field] is not None:
            if isinstance(row[field], list):
                diagnoses.extend(row[field])
            else:
                diagnoses.append(str(row[field]))
    
    # Filter for respiratory codes only using correct parsing
    resp_codes = []
    for d in diagnoses:
        if d and isinstance(d, str):
            d_clean = d.strip().upper()
            # Check J09-J98
            if d_clean.startswith('J'):
                match = re.match(r'J(\d{2})', d_clean)
                if match and 9 <= int(match.group(1)) <= 98:
                    resp_codes.append(d)
            # Check I26-I27
            elif d_clean.startswith('I'):
                match = re.match(r'I(\d{2})', d_clean)
                if match and 26 <= int(match.group(1)) <= 27:
                    resp_codes.append(d)
    
    print(f"  Participant {row['eid']}: {', '.join(resp_codes[:10])}")  # Show first 10 codes

# Save filtered cohort to CSV
output_csv = 'ukb_respiratory_cohort.csv'
respiratory_df.to_csv(output_csv, index=False)
print(f"\nFiltered respiratory cohort saved to: {output_csv}")

# Also save full dataset (unfiltered) for comparison
output_csv_full = 'ukb_full_data.csv'
df.to_csv(output_csv_full, index=False)
print(f"Full dataset saved to: {output_csv_full}")


## Analyze ICD-10 Code Distribution in Respiratory Cohort

Let's examine the distribution of respiratory ICD-10 codes to understand the types of respiratory diseases in our cohort.


In [ ]:
from collections import Counter

# Collect all respiratory ICD-10 codes from the cohort
all_resp_codes = []

for idx, row in respiratory_df.iterrows():
    for field in ['p41202', 'p41204', 'p41270']:
        if field in row.index and row[field] is not None:
            if isinstance(row[field], list):
                codes = row[field]
            else:
                codes = [str(row[field])]
            
            # Filter for respiratory codes (J09-J98, I26-I27)
            for code in codes:
                if code and isinstance(code, str):
                    code = code.strip().upper()
                    # Check if it's a respiratory code
                    # Use correct ICD-10 parsing: extract first 2 digits (category)
                    if code.startswith('J'):
                        match = re.match(r'J(\d{2})', code)
                        if match:
                            category = int(match.group(1))
                            if 9 <= category <= 98:
                                all_resp_codes.append(code)
                    elif code.startswith('I'):
                        match = re.match(r'I(\d{2})', code)
                        if match:
                            category = int(match.group(1))
                            if 26 <= category <= 27:
                                all_resp_codes.append(code)

# Count frequency of each code
code_counts = Counter(all_resp_codes)

print(f"Total respiratory diagnoses recorded: {len(all_resp_codes)}")
print(f"Unique respiratory diagnosis codes: {len(code_counts)}")
print(f"\nTop 20 most common respiratory diagnoses:")
print("-" * 60)

for code, count in code_counts.most_common(20):
    percentage = (count / len(respiratory_df)) * 100
    print(f"{code:8s} : {count:6d} cases ({percentage:5.2f}% of cohort)")

# Breakdown by major category
j_codes = {k: v for k, v in code_counts.items() if k.startswith('J')}
i_codes = {k: v for k, v in code_counts.items() if k.startswith('I')}

print(f"\n\nBreakdown by ICD-10 category:")
print(f"J codes (Respiratory system diseases): {sum(j_codes.values())} diagnoses across {len(j_codes)} unique codes")
print(f"I26-I27 codes (Pulmonary heart disease): {sum(i_codes.values())} diagnoses across {len(i_codes)} unique codes")

# Save code statistics to file
import json
code_stats = {
    'total_participants': len(respiratory_df),
    'total_diagnoses': len(all_resp_codes),
    'unique_codes': len(code_counts),
    'top_20_codes': [{'code': code, 'count': count, 'percentage': round((count/len(respiratory_df))*100, 2)} 
                     for code, count in code_counts.most_common(20)],
    'category_breakdown': {
        'J_codes': {'count': sum(j_codes.values()), 'unique': len(j_codes)},
        'I_codes': {'count': sum(i_codes.values()), 'unique': len(i_codes)}
    }
}

stats_file = 'icd10_code_statistics.json'
with open(stats_file, 'w') as f:
    json.dump(code_stats, f, indent=2)
    
print(f"\n\nCode statistics saved to: {stats_file}")


## Retrieve data from the table

The following code selects the `participant` table.
Then we can define which field we are interested in using the `find_field` function.

There are three main ways to identify the field of interest:

- With `name` argument: here we give field ID. We can construct field ID used by the `dxdata` package from the field ID defined by UKB Showcase. The numeric showcase ID is translated to the Spark DB column name by adding the letter `p` at the beginning: e.g. *Standing height* showcase id is `50`, so Spark ID would be `p50`. Usually, fields have multiple instances. In such case, we add the `_i` suffix followed by instance number, e.g. *Standing height | Instance 0* will be `p50_i0`
- With `title` argument: here we define the field by full title, followed by ` | Instance` suffix, e.g. `Age at recruitment` or `Standing height | Instance 0`
- With `title_regex` argument: here we define the field by [regular expression](https://docs.python.org/3/howto/regex.html) matching the part of the title. We can use a keyword here, e.g. `.*height.*` will return all columns with the word *height* in the title.

In [3]:
pheno = dataset['participant']

# Find by field name
field_eid = pheno.find_field(name="eid")

# Find by exact title
field_sex = pheno.find_field(title="Sex")
field_age = pheno.find_field(title="Age at recruitment")
field_height = pheno.find_field(title="Standing height | Instance 0")

# Find by title pattern
pattern = ".*height.*"
fields_height = list(pheno.find_fields(title_regex=pattern))

In [4]:
for _ in fields_height:
    print("[" + _.name + "]\t" + _.title + " (" + _.linkout + ")")
    

[p50_i0]	Standing height | Instance 0 (http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=50)
[p50_i1]	Standing height | Instance 1 (http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=50)
[p50_i2]	Standing height | Instance 2 (http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=50)
[p50_i3]	Standing height | Instance 3 (http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=50)
[p51_i0]	Seated height | Instance 0 (http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=51)
[p51_i1]	Seated height | Instance 1 (http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=51)
[p51_i2]	Seated height | Instance 2 (http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=51)
[p51_i3]	Seated height | Instance 3 (http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=51)
[p1697_i0]	Comparative height size at age 10 | Instance 0 (http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=1697)
[p1697_i1]	Comparative height size at age 10 | Instance 1 (http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=1697)
[p1697_i2]	Comparative height size